In [1]:
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import numpy as np
import pylab as plt
import tensorflow as tf
from tensorflow.python.keras._impl.keras.layers.recurrent import LSTM

Using TensorFlow backend.


In [2]:
import os,glob
folder_path = 'label_02'
allexamples=[]
lengths = [] #keep track of how many lines are in each sequence
for filename in glob.glob(os.path.join(folder_path, '*.txt')):
    with open(filename, 'r') as f:
        x = f.readlines()
        allexamples.append(x)
        lengths.append(len(x))

In [3]:
#make it more readable
newlist = []
for i in range(0,len(allexamples)):
    for j in allexamples[i]:
        innerlist = j.split()
        withidentifier = [i] + innerlist #so we know which sequence these belong to.
        newlist.append(withidentifier)

In [4]:
import pandas as pd
df = pd.DataFrame.from_records(newlist)

In [5]:
biglist= []
testone = [] #TEST WITH ONLY X to see if that works
for i in range(0,65301):#length of df
    smalllist = []
    smalltest = []
    if df.iloc[i][3] != 'DontCare':
        x1 = float(df.iloc[i][7])
        x2 = float(df.iloc[i][9])
        y1 = float(df.iloc[i][8])
        y2 = float(df.iloc[i][10])
        x = (x1+x2)/2
        y = (y1+y2)/2
        smalllist.extend([int(df.iloc[i][0]),int(df.iloc[i][1]),x,y]) #[i][0]  =sequence no. [i][1]=frame no
        smalltest.extend([int(df.iloc[i][0]),int(df.iloc[i][1]),x])
    if len(smalllist) != 0:
        biglist.append(smalllist)
    if len(smalltest) != 0:
        testone.append(smalltest)

In [6]:
interlist = []
middlelists = []
sequencelist = []
for i in range(0,47262):
    
    if biglist[i][0] == biglist[i-1][0] and biglist[i][1] == biglist[i-1][1]:
        middlelists.append([round(biglist[i][2]),round(biglist[i][3])])
    
    elif biglist[i][0] == biglist[i-1][0] and biglist[i][1] != biglist[i-1][1]:
        sequencelist.append(middlelists)
        middlelists = []
        middlelists.append([round(biglist[i][2]),round(biglist[i][3])])
    else:
        sequencelist.append(middlelists)
        interlist.append(sequencelist)
        sequencelist = []
        middlelists = []
        middlelists.append([round(biglist[i][2]),round(biglist[i][3])])

In [7]:
for seq in interlist:
    for frame in seq:
        if len(frame) > 3:
            n = len(frame) - 3
            del frame[-n:]
        else:  ##NEED to make sure that list is shaped regularly, ie that each frame has 3 elements
            #need 'don't care' but not sure how?? just put [0,0] for now, need to normalize it also
            n = 3 - len(frame)
            for j in range(0,n):
                frame.append([0,0]) 

In [8]:
del interlist[0] #remove empty list at top

In [9]:
#for x and y

allsequencesteps = []
for sequence in interlist:
    
    seqsteps = [sequence[i:i + 60] for i in range(0, len(sequence), 60)]
    
    if len(seqsteps[-1]) != 60:
        del seqsteps[-1]
        allsequencesteps.append(seqsteps)
    else:
        allsequencesteps.append(seqsteps)
#need to reshape all sequences into sub sequences of 60 frames.
#but all sequences have different lengths.
#just remove any set of frames that don't quite make up 60 frames at the end.
    

In [10]:
#removing sequences that have only one set of 60 frames
#no use for training because don't have next 60 frames to test against
indexno = []
for i in range (0,19):
    
    if len(allsequencesteps[i]) < 2:
        indexno.append(i)

In [11]:
for number in indexno:
    del allsequencesteps[number]  #remove sequences that are too short

In [12]:
#18 sequences left that had more than 1 60 frame section
#some sequences only have two sets of 60 frames
#change all sequences to only first two so they are all same shape
#if this works maybe we can just split up sequences into chunks
#and train separately

#this is actually the only way this will work with three sets of coordinates per frame
#because then x and y are shaped (1,60,3,2) ie one set of 60 frames. for all of the sequences
#if train on more sets, LSTM cannot take that as an input as it is 4d

#i tried leaving the sequences with as many sets of 60 as they had, for example the first one
#allsequencesteps[0] had 11, and I tried training with just this sequence but bc shape was
#(11,60,3,2) it couldn't take this as an input.

for i in range(0,17):
    diff = len(allsequencesteps[i]) - 2
    del allsequencesteps[i][-diff:]

In [14]:
len(allsequencesteps[0]) #all sequences have length 2 (2 sets of 60 frames)

2

In [15]:
#x and y normalisation

#I CAN'T GET THIS TO DO IT CORRECTLY think i'm just getting confused.
#all sequences end up having the same numbers, i think it just does the first sequence.
from sklearn import preprocessing
normalized_Xy = []
framenorms = []
setnorm = []
for sequence in allsequencesteps:
    for setframes in sequence: 
        for singleframes in setframes:
            normframe = preprocessing.normalize(singleframes) 
            framenorms.append(normframe)
        
        setnorm.append(framenorms)
    normalized_Xy.append(setnorm)
    

In [20]:
normalized_Xy = np.array(normalized_Xy)
normalized_Xy.shape   #this aint right

(18, 33, 1980, 3, 2)

In [26]:
#for now just showing the training on not normalized data while i figure out 
#what i'm doing wrong

xseq = []
yseq = []
for k in range(0,17):

    xmidseq = allsequencesteps[k][:-1]
    ymidseq = allsequencesteps[k][1:]
    xseq.append(xmidseq)
    yseq.append(ymidseq)

In [28]:
xtrnorm = xseq[0:17]
ytrnorm = yseq[0:17]
xtenorm = xseq[17:18] #save last sequence to test.
ytenorm = yseq[17:18]

In [32]:
len(xtrnorm[0]) #only one set of 60 frames for x and y

1

In [31]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Bidirectional

In [33]:
model2 = Sequential()
model2.add(LSTM(2, return_sequences = True,input_shape=(3, 2)))
model2.add(Dense(2))
model2.compile(optimizer='adam', loss='mse')

In [34]:
#not good results because it isn't normalised.
#would also need to train the model on one set of 60 frames at a time, if this is possible
history = model2.fit(xtrnorm[0], ytrnorm[0], epochs=500, validation_split=0.2, verbose=1)

Train on 48 samples, validate on 12 samples
Epoch 1/500
48/48 [==============================] - 1s 11ms/step - loss: 256161.3438 - val_loss: 411606.6562
Epoch 2/500
48/48 [==============================] - 0s 868us/step - loss: 256156.6094 - val_loss: 411599.1562
Epoch 3/500
48/48 [==============================] - 0s 530us/step - loss: 256151.7708 - val_loss: 411591.8438
Epoch 4/500
48/48 [==============================] - 0s 962us/step - loss: 256147.5104 - val_loss: 411585.8750
Epoch 5/500
48/48 [==============================] - 0s 724us/step - loss: 256143.3802 - val_loss: 411580.2188
Epoch 6/500
48/48 [==============================] - 0s 513us/step - loss: 256139.2865 - val_loss: 411574.5000
Epoch 7/500
48/48 [==============================] - 0s 385us/step - loss: 256135.0521 - val_loss: 411568.8438
Epoch 8/500
48/48 [==============================] - 0s 372us/step - loss: 256131.0312 - val_loss: 411563.0000
Epoch 9/500
48/48 [==============================] - 0s 412us/step - 